<a href="https://colab.research.google.com/github/gopiBNK/FAANG/blob/main/FAANG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

st.title("FAANG Stock Price Predictor")

df = pd.read_csv("faang.csv")

st.subheader("Dataset Preview")
st.dataframe(df.head())

X = df[["Ticker", "Open", "High", "Low", "Volume"]]
y = df["Close"]

# Train & Log Model
@st.cache_resource
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    preprocess = ColumnTransformer([
        ("ticker", OneHotEncoder(handle_unknown="ignore"), ["Ticker"])
    ], remainder="passthrough")

    model = Pipeline([
        ("preprocess", preprocess),
        ("reg", LinearRegression())
    ])

    mlflow.set_experiment("FAANG_Stock_Price_Prediction")

    with mlflow.start_run():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        mlflow.log_param("model", "LinearRegression")
        mlflow.log_param("test_size", 0.2)

        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2_Score", r2)

        mlflow.sklearn.log_model(model, "model")

    return model, mae, rmse, r2, y_test, y_pred

model, mae, rmse, r2, y_test, y_pred = train_model(X, y)

# Metrics Display
st.subheader("Model Performance")

col1, col2, col3 = st.columns(3)
col1.metric("MAE", f"{mae:.2f}")
col2.metric("RMSE", f"{rmse:.2f}")
col3.metric("R² Score", f"{r2:.3f}")

# Visualization
st.subheader("Actual vs Predicted Close Price")

fig = plt.figure()
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Close Price")
plt.ylabel("Predicted Close Price")
plt.title("Actual vs Predicted")
st.pyplot(fig)
plt.close()

# User Input
st.subheader("🔍 Enter Stock Details")

ticker = st.text_input("Ticker (AAPL, AMZN, META, GOOG, NFLX)")
open_price = st.number_input("Open Price", min_value=0.0, format="%.2f")
high_price = st.number_input("High Price", min_value=0.0, format="%.2f")
low_price = st.number_input("Low Price", min_value=0.0, format="%.2f")
volume = st.number_input("Volume", min_value=0)

if st.button("Predict Close Price"):
    if ticker.strip() == "":
        st.error("Please enter a ticker value")
    else:
        input_df = pd.DataFrame([{
            "Ticker": ticker.upper(),
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Volume": volume
        }])

        prediction = model.predict(input_df)[0]
        st.success(f"Predicted Close Price: {prediction:.2f}")
